# Exercise: Quantum Annealing for Sampling Ionic Distributions in Disordered Materials

### Disordered Materials in Energy Research

In energy research, particularly within the domain of battery technologies, the investigation of disordered materials, such as doped materials and high-entropy oxides, has become increasingly significant. A hallmark of this disorder is the presence of vacancies alongside ions. The distribution between ions and vacancies is far from trivial. Consider the commonly used cathode material in lithium-ion batteries: $\mathrm{LiCoO_2}$. The figure below illustrates this with a total of 17 occupied Li sites, representing the charged state with 100% lithiation. During battery discharge, lithium is extracted from the material and the lithium content decreases. To study its properties, for instance, at a decreased lithiation of 70%, we need to allocate 12 Li ions across 17 sites. The figure depicts two of the myriad potential configurations. The specific arrangement of ions and vacancies critically impacts the material's properties. This distribution is especially crucial in computational studies, where the accuracy of predictions hinges upon an accurate representation of ion-vacancy configurations. Given the extensive possible configurations, ensuring model precision is challenging; inaccuracies can substantially alter predictive results.

<div style="text-align: center">
    <img src="LCO.jpg" width="500"/>
</div>

Thermodynamically, the lowest energy configurations are the most significant. Materials are inherently inclined to adopt these configurations, making them critical for both understanding and predicting the behavior of the material under various conditions. The challenge, however, lies in effectively identifying these ground-state distributions from a vast configuration space. Quantum Annealing presents an advanced approach to efficiently sample ionic distributions in disordered materials.

This tutorial delves into the specifics of Quantum Annealing, elucidating its capabilities and techniques for identifying low-energy states in disordered materials.

### Simplified Model: Classical Electrostatic Coulomb Energy

Coulomb interactions are the most fundamental interactions in ionic systems. In the simplest Coulomb model, the energy $E_{\mathrm{coul}}$ is determined by the summation of pairwise interactions between every distinct pair of ions in the system. In the equation:

$$E_{\mathrm{coul}} = \sum_{i<j} \frac{q_i\,q_j}{|r_i-r_j|}$$

$q_i$ and $q_j$ represent the charges of the ions, while $|\mathbf{r}_i-\mathbf{r}_j|$ is the distance between the ions $i$ and $j$. For the sake of simplicity, we have set all units to one. This equation encapsulates the essence of classical electrostatic interactions: charged entities attract or repel each other based on the inverse of the square of their separation distance. While this is a simplification and does not account for quantum effects or other complexities, it serves as a foundational starting point for understanding the energetics of ionic systems in disordered materials.

## Problem 1: Electrostatic 2D Lattice

For this investigation, we focus on a model representing a two-dimensional lattice subjected to classical electrostatic Coulomb interactions. This model represents a **hypothetical** material $\mathrm{Li_2WO_4}$ for a Li-ion battery wherein two lithium (Li) ions and two vacancies are distributed over a set of four available lattice sites for lithium.

The visual representation below illustrates the lattice alongside specific ionic positions. The accompanying table enumerates the exact atomic positions. Note that only two out of the four available lithium sites are occupied for any given configuration.

<div style="text-align: center">
    <img src="2D_lattice.jpg" width="500"/>
</div>


#### **_Task 1.1: Complete the following Table with the coordinates of each atom_**

| Site Number | Atom      | x-Coordinate | y-Coordinate |
|-------------|-----------|--------------|--------------|
| 1           | Li<sup>+</sup> | 0            | 0            |
| 2           | Li<sup>+</sup> | ---          | ---          |
| 3           | Li<sup>+</sup> | ---          | ---          |
| 4           | Li<sup>+</sup> | 2            | 1            |
| 5           | O<sup>2-</sup> | ---          | ---          |
| 6           | O<sup>2-</sup> | ---          | ---          |
| 7           | O<sup>2-</sup> | ---          | ---          |
| 8           | O<sup>2-</sup> | ---          | ---          |
| 9           | W<sup>6+</sup> | 1            | 0.5          |

##### _Solution:_

| Site Number | Atom      | x-Coordinate | y-Coordinate |
|-------------|-----------|--------------|--------------|
| 1           | Li<sup>+</sup> | 0            | 0            |
| 2           | Li<sup>+</sup> | 2            | 0            |
| 3           | Li<sup>+</sup> | 0            | 1            |
| 4           | Li<sup>+</sup> | 2            | 1            |
| 5           | O<sup>2-</sup> | 1            | 0            |
| 6           | O<sup>2-</sup> | 0            | 0.5          |
| 7           | O<sup>2-</sup> | 2            | 0.5          |
| 8           | O<sup>2-</sup> | 1            | 1            |
| 9           | W<sup>6+</sup> | 1            | 0.5          |

### Computing the Coulomb energies of all possible configurations

The process of computing the Coulomb energy for a set of ionic configurations in our 2D lattice model hinges on the Classical Electrostatic Coulomb Energy equation:
$$E_{\mathrm{coul}}\ =\ \sum_{i<j} \frac{q_i\,q_j}{|\mathbf{r}_i-\mathbf{r}_j|}$$
The formula calculates the pairwise Coulomb interaction energy between ions in the system. The energy between two ions is proportional to the product of their charges and inversely proportional to the distance between them. Here's how we can use this to evaluate the energy for our lattice:

1. **Pairwise Computation**: For every distinct pair of ions, we will compute their interaction energy.
2. **Accumulation**: Sum all these pairwise energies to get the total Coulombic energy for a given ionic configuration.
3. **Iterate for All Configurations**: By varying the arrangement of Li ions and vacancies, we can compute the energy for each unique configuration.

Let us denote any given Li distribution by the pair of occupied sites, e.g., Configuration 1: (1,3), meaning that one Li-ion is sitting on site number 1 and the other Li-ion on site number 3 (see the Table above for site numbers).

#### **_Task 1.2: Complete the following Table with the Coulomb energies of all possible Li ion configurations_**

| Configuration | Coulomb energy | Binary Representation | Coulomb energy from QUBO |
| --- | --- | --- | --- |
| (1,2) | -57.225 | [1,1,0,0] | -57.225 |
| --- | --- | --- | --- |
| --- | --- | --- | --- |
| --- | --- | --- | --- |
| --- | --- | --- | --- |
| --- | --- | --- | --- |

_Fill in the first two columns of the above Table using the following script._

In [1]:
import numpy as np

def energy_coul_pair(site1, site2):
    """Compute Coulomb energy between two sites."""
    q1 = site1[0]  # Charge of the first ion
    r1 = np.array(site1[1])  # Position of the first ion
    q2 = site2[0]  # Charge of the second ion
    r2 = np.array(site2[1])  # Position of the second ion
    
    # Compute pairwise interaction energy
    return q1 * q2 / np.linalg.norm(r1 - r2)

# Define the ionic sites and their charges/positions.
# Each site is represented as a tuple: (charge, [x,y])

site_1 = (1, [0, 0])
site_2 = (1, [2, 0])
site_3 = (1, [0, 1])
site_4 = (1, [2, 1])

#select two occupied Li sites and find all possible configurations
li_sites_occupied = [site_1, site_2] 

o_sites = [(-2, [1, 0]), (-2, [0, 0.5]), (-2, [2, 0.5]), (-2, [1, 1])]
w_sites = [(6, [1, 0.5])]

# Combine all sites
all_sites = li_sites_occupied + o_sites + w_sites
n_all_sites = len(all_sites)

# Initialize total energy to zero
energy_coul_total = 0
# Loop over any distinct pair of sites (both Li and other species)
for i in range(n_all_sites):
    for j in range(i+1, n_all_sites):  # Ensures we don't double-count pairs
        energy_coul_total += energy_coul_pair(all_sites[i], all_sites[j])

print('Total Coulomb energy for the configuration: ' + str(round(energy_coul_total,3)))


Total Coulomb energy for the configuration: -57.225


##### _Solution:_

| Configuration | Coulomb energy | Binary Representation | Coulomb energy from QUBO |
| --- | --- | --- | --- |
| (1,2) | -57.225 | [1,1,0,0] | -57.225 |
| (1,3) | -56.725 | [1,0,1,0] | -56.725 |
| (1,4) | -57.278 | [1,0,0,1] | -57.278 |
| (2,3) | -57.278 | [0,1,1,0] | -57.278 |
| (2,4) | -56.725 | [0,1,0,1] | -56.725 |
| (3,4) | -57.225 | [0,0,1,1] | -57.225 |

### Mapping the Configurational Coulomb Energy onto QUBO Cost Function

Mapping the Coulomb energy into a QUBO form is crucial for processing on quantum annealers. As we learned earlier, a QUBO problem seeks to minimize a function of binary variables (variables that take on values 0 or 1). We can naturally map our problem of minimizing the Coulomb energy as a function of Li-ion distribution to a QUBO. For this, we introduce for each Li-ion site $i$ a binary variable $x_i$, which represents whether the site is occupied ($x_i = 1$) or vacant ($x_i = 0$). We can then write $E_{coul}$ in the following QUBO form:

$$E_{coul} = c + \sum_{i} \alpha_i x_i +\sum_{i<j} \beta_{ij} x_i x_j$$

The individual terms in this expression are defined as follows:

1. **Constant Term, $c$**: This is the Coulomb energy arising solely from the fixed ions (like $O^{2-}$ and $W^{6+}$). Since these positions and charges are fixed, this energy doesn't change with different configurations and can be computed just once. 

2. **Linear Terms, $\sum_{i} \alpha_i x_i$**: Each linear term represents the interaction energy between a given site with variable occupation (in our example a Li site) and all the fixed ions in the lattice. The coefficients $\alpha_i$ thus capture the cumulative interaction of the $i$-th variable site with all fixed ions.

3. **Quadratic Terms, $\sum_{i<j} \beta_{ij} x_i x_j$**: This term captures the pairwise interaction energies between all possible pairs of variable sites, i.e., any Li-Li interaction term. The coefficients $\beta_{ij}$ correspond to the interaction energy between the *i*-th and *j*-th variable sites. Note that the interaction energy of a given pair of sites only contributes to the total energy if both sites are occupied (i.e., both $x_i = 1$ and $x_j =1 $). 

This decomposition makes the energy expression amenable to quantum annealing, by breaking it down into terms that rely on binary decision variables. We note that the constant term $c$ does not have any influence on the minimization problem and thus can be ignored. By defining $Q_{i,i} = \alpha_i$ and $Q_{i,j} = \beta_{ij}$, we then obtain the Coulomb energy precisely in the QUBO form introduced earlier:

$$ E_{coul}(\{x_i\})\ =\ c + \sum_{i = 1}^n \,Q_{i,i}\,x_i + \sum_{i<j = 1}^n \,Q_{i,j}\,x_i\,x_j$$



#### **_Task 1.3: Compute the individual coefficients_**

In this Task, you will calculate the individual coefficients of the Coulomb energy in the QUBO form discussed above.

- **Compute $\alpha_i$**: For each Li site, compute its interaction energy with all fixed ions. This gives the $\alpha_i$ values.
  
- **Compute $\beta_{ij}$**: For every pair of Li sites, compute their interaction energy. These are the $\beta_{ij}$ values.

- **Compute $c$**: This involves evaluating the cumulative interaction energy between all fixed ions.

_Complement the following script to compute the vector $\vec{\alpha} = (\alpha_1, \alpha_2, ...)$, matrix $\hat{\beta} = (\beta_{ij})$, and constant $c$ for the 2D lattice_

In [2]:
# Compute c: Coulomb energy from fixed atoms
def compute_c(fixed_sites):
    c_energy = 0
    for i in range(len(fixed_sites)):
        for j in range(i+1, len(fixed_sites)):  # Ensures we don't double-count pairs
            c_energy += energy_coul_pair(fixed_sites[i], fixed_sites[j])
    return c_energy

# Compute alpha vector: The Coulomb energies of any given Li site with all fixed ions
def compute_alpha(li_sites, fixed_sites):
    alpha = []
    for li in li_sites:
        alpha_energy = 0
        for site in fixed_sites:
            alpha_energy += energy_coul_pair(li, site)
        alpha.append(alpha_energy)
    return np.array(alpha)

# Compute beta: The Coulomb energies of all pairs of Li sites
def compute_beta(li_sites):
    beta = np.zeros((len(li_sites), len(li_sites)))
    for i in range(len(li_sites)):
        for j in range(i+1, len(li_sites)):
            beta[i, j] = energy_coul_pair(li_sites[i], li_sites[j])
    return beta

# Compute the QUBO parameters
li_sites_all = [site_1, site_2, site_3, site_4]
fixed_sites = o_sites + w_sites
c = compute_c(fixed_sites)
alpha = compute_alpha(li_sites_all, fixed_sites)
beta = compute_beta(li_sites_all)

# Print the QUBO parameters
print("c =", c)
print("alpha =", alpha)
print("beta matrix =")
print(beta)

c = -51.68916494400135
alpha = [-3.01779292 -3.01779292 -3.01779292 -3.01779292]
beta matrix =
[[0.        0.5       1.        0.4472136]
 [0.        0.        0.4472136 1.       ]
 [0.        0.        0.        0.5      ]
 [0.        0.        0.        0.       ]]


#### **_Task 1.4: Recompute the Coulomb energies of all possible configurations using the QUBO form_**

_Fill in the last two columns of the Table in Task 1.1 above. First fill in all binary representations of the different configurations. Then use the following script to compute the Coulomb energy for each binary representation._

In [3]:
# Calculate the QUBO Coulomb energy for a Li configuration x = [x_0, x_1, ...]
def E_QUBO(x, alpha, beta, c):
    E = c
    for i in range(len(x)):
        E += alpha[i] * x[i]
        for j in range(i+1, len(x)):
            E += beta[i, j] * x[i] * x[j]
    return E

# Provide a configuration
x = [1,1,0,0]

E = E_QUBO(x, alpha, beta, c)
print(f"Coulomb energy from QUBO for x = {x} is {round(E,3)}")

Coulomb energy from QUBO for x = [1, 1, 0, 0] is -57.225


**_QUESTION: What is the lowest energy (ground state) configuration?_**

### Finding the ground state using the D-Wave Quantum Annealer

Now that we have formulated the Coulomb energy in QUBO form and calculated the QUBO coefficients, we will use the D-Wave Quantum Annealer to find the ground state configuration of the 2D lattice "$\mathrm{Li_2WO_4}$". So let's reuse the basic D-Wave sampling script...

In [4]:
from dwave.system import DWaveSampler, EmbeddingComposite
import dwave.inspector

import numpy as np


def get_token():
    return 'COPY YOUR PERSONAL TOKEN HERE'


_Use the definition $Q_{i,i} = \alpha_i$ and $Q_{i,j} = \beta_{ij}$ to define the function "get_qubo"._

In [9]:
def get_qubo(alpha, beta):

    dim = len(alpha)
    
    Q = {}
    for i in range(dim):
        Q[(i, i)] = Q.get((i, i), 0) + alpha[i]
        for j in range(dim):
            Q[(i, j)] = Q.get((i, j), 0) + beta[i][j]

    return Q

Q = get_qubo(alpha, beta)

print(Q)

{(0, 0): -3.0177929165189328, (0, 1): 0.5, (0, 2): 1.0, (0, 3): 0.4472135954999579, (1, 1): -3.0177929165189328, (1, 0): 0.0, (1, 2): 0.4472135954999579, (1, 3): 1.0, (2, 2): -3.017792916518931, (2, 0): 0.0, (2, 1): 0.0, (2, 3): 0.5, (3, 3): -3.017792916518931, (3, 0): 0.0, (3, 1): 0.0, (3, 2): 0.0}


We are now ready to search the ground state configuration of the 2D lattice of "$\mathrm{Li_2WO_4}$".

In [10]:
def sample_qubo(Q, nruns):

    # We choose the quantum annealer as solver for our QUBO problem
    sampler = EmbeddingComposite(DWaveSampler())

    # And we make nruns independent annealing runs...
    sample_set = sampler.sample_qubo(Q, num_reads=nruns)

    print(sample_set)

    # Print the correct Coulomb energies

    nVar = len(alpha)

    for sample in sample_set:
        sample_list = nVar*[0]
        for key in sample:
            sample_list[key] = sample[key]
        print('Sample: ' + str(sample_list))
        print('Energy: ' + str(E_QUBO(sample_list, alpha, beta, c)))
        print('N Li: ' + str(sum(sample_list)))

    return sample_set

nruns = 10
sample_set = sample_qubo(Q, nruns)

   0  1  2  3    energy num_oc. chain_.
0  1  1  1  1 -8.176744      10     0.0
['BINARY', 1 rows, 10 samples, 4 variables]
Sample: [1, 1, 1, 1]
Energy: -59.865909419077155
N Li: 4


_Careful_: Since the constant term is omitted in the D-Wave QUBO, the "energies" in the sample set do not directly correspond to the full Coulomb energy. For this, we added a loop over the samples and recompute the full energy for each.

#### **_Task 1.5: Examine the solution_**

_What do you observe? Did you obtain the ground state that you already identified earlier "by hand"? How can you explain the D-Wave result? Is it the lowest energy solution?_

### Enforcing the correct Li ion number using constraints

Each site occupation variable can be either zero or one. The total number of Li ions is given by 

$$ N_{\mathrm{Li}} = \sum_i x_i $$

Depending on the values of $x_i$, the total Li number therefore can change. In our physical problem, we want to have precisely two Li ions in the lattice, i.e., $N_{\mathrm{Li}}^{\mathrm{target}} = 2$. We must find a way to tell this to the D-Wave Annealer...

The basic idea is to put an energy penalty on all states that violate this constraint. For configurations with the correct target Li number, the energy penalty should be zero. We will then simply add the energy penalty to the QUBO of the Coulomb problem. To be able to do this, the energy penalty itself must be a quadratic function of the binary variables. These conditions are met by the following energy penalty:

$$ E_{\mathrm{penalty}} = \lambda\,\left( \sum_i x_i - N_{\mathrm{Li}}^{\mathrm{target}} \right)^2 $$

Here, $\lambda$ is a parameter that allows us to tune how strongly the number constraint is enforced.

#### **_Task 1.6: Rewrite the energy penalty in QUBO form_**

_Factor out the squared expression for the energy penalty function to obtain the corresponding QUBO coefficients in the form_

$$ E_{\mathrm{penalty}}(\{x_i\})\ =\ c + \sum_{i = 1}^n \,Q^{\mathrm{p}}_{i,i}\,x_i + \sum_{i<j = 1}^n \,Q^{\mathrm{p}}_{i,j}\,x_i\,x_j$$

Solution:
$$ E_{\mathrm{penalty}} = \lambda\,\left(N_{\mathrm{Li}}^{\mathrm{target}}\right)^2 + \sum_i \lambda\left(1-2\,N_{\mathrm{Li}}^{\mathrm{target}}\right)\,x_i + \sum_{i<j} 2\lambda\,x_i\,x_j $$

We thus find $Q^{\mathrm{p}}_{i,i} = \lambda\left(1-2\,N_{\mathrm{Li}}^{\mathrm{target}}\right)$ and $Q^{\mathrm{p}}_{i,j} = 2\lambda$. The constant term can, again, be ignored since it does not affect the minimization solution.

Let's add the penalty coefficients to our QUBO...

In [15]:
def add_penalty_qubo(Q, lam, Ntar, nVar):

    for i in range(nVar):
        Q[(i, i)] = Q.get((i, i), 0) + lam * (1 - 2 * Ntar)
        for j in range(i + 1, nVar):
            Q[(i, j)] = Q.get((i, j), 0) + lam * 2

    return Q

lam = 1
Ntar = 2
nVar = len(alpha)

Q = get_qubo(alpha, beta)
Q = add_penalty_qubo(Q, lam, Ntar, nVar)

nruns=100
sample_set = sample_qubo(Q, nruns)

   0  1  2  3     energy num_oc. chain_.
0  1  1  1  0 -10.106165      33     0.0
1  1  1  0  1 -10.106165      22     0.0
3  1  0  1  1 -10.106165      23     0.0
2  0  1  1  1 -10.106165      15     0.0
4  1  0  0  1  -9.588372       2     0.0
5  0  1  1  0  -9.588372       1     0.0
6  1  1  0  0  -9.535586       3     0.0
7  1  0  1  0  -9.035586       1     0.0
['BINARY', 8 rows, 100 samples, 4 variables]
Sample: [1, 1, 1, 0]
Energy: -58.79533009805819
N Li: 3
Sample: [1, 1, 0, 1]
Energy: -58.79533009805819
N Li: 3
Sample: [1, 0, 1, 1]
Energy: -58.79533009805818
N Li: 3
Sample: [0, 1, 1, 1]
Energy: -58.79533009805818
N Li: 3
Sample: [1, 0, 0, 1]
Energy: -57.27753718153925
N Li: 2
Sample: [0, 1, 1, 0]
Energy: -57.27753718153925
N Li: 2
Sample: [1, 1, 0, 0]
Energy: -57.22475077703922
N Li: 2
Sample: [1, 0, 1, 0]
Energy: -56.72475077703921
N Li: 2


In [35]:
def add_mu_qubo(Q, mu, nVar):

    for i in range(nVar):
        Q[(i, i)] = Q.get((i, i), 0) - mu

    return Q

mu = -2
lam = 0
Ntar = 2
nVar = len(alpha)

Q = get_qubo(alpha, beta)
Q = add_penalty_qubo(Q, lam, Ntar, nVar)
Q = add_mu_qubo(Q, mu, nVar)

nruns=100
sample_set = sample_qubo(Q, nruns)

   0  1  2  3    energy num_oc. chain_.
0  0  1  1  0 -1.588372      43     0.0
1  1  0  0  1 -1.588372      27     0.0
2  1  1  0  0 -1.535586       9     0.0
3  0  0  1  1 -1.535586      21     0.0
['BINARY', 4 rows, 100 samples, 4 variables]
Sample: [0, 1, 1, 0]
Energy: -57.27753718153925
N Li: 2
Sample: [1, 0, 0, 1]
Energy: -57.27753718153925
N Li: 2
Sample: [1, 1, 0, 0]
Energy: -57.22475077703922
N Li: 2
Sample: [0, 0, 1, 1]
Energy: -57.224750777039205
N Li: 2
